In [133]:
import sys
# sys.path.insert(0,'Repos/newcastle/air-quality/src/')
sys.path.insert(0,'../src')
from v0 import *
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, explained_variance_score

SHAPE_FOLDER = '/home/adelsondias/Repos/newcastle/air-quality/shape/Middle_Layer_Super_Output_Areas_December_2011_Full_Extent_Boundaries_in_England_and_Wales/'
DATA_FOLDER = '/home/adelsondias/Repos/newcastle/air-quality/data_30days/'
zx, zi = load_data__(DATA_FOLDER)
print('data loaded')

freq='D'
variables =  {
'sensors':['NO2','Temperature','O3','CO'],
    'exogenous':['primary','trunk','motorway','traffic_signals','day','dow','hour'] 
}
if freq=='D':
    variables['exogenous'] = ['primary','trunk','motorway','traffic_signals','day','dow'] 

zx, zi = resampling_sensors__(zx, zi, freq)

data loaded


In [137]:
zx.columns

Index(['NO2', 'NO2', 'NO2', 'NO2', 'NO2', 'd_NO2', 'd_NO2', 'd_NO2', 'd_NO2',
       'd_NO2', 'Temperature', 'Temperature', 'Temperature', 'Temperature',
       'Temperature', 'd_Temperature', 'd_Temperature', 'd_Temperature',
       'd_Temperature', 'd_Temperature', 'O3', 'O3', 'O3', 'O3', 'O3', 'd_O3',
       'd_O3', 'd_O3', 'd_O3', 'd_O3', 'CO', 'CO', 'CO', 'CO', 'CO', 'd_CO',
       'd_CO', 'd_CO', 'd_CO', 'd_CO', 'dow', 'day', 'primary', 'trunk',
       'motorway', 'traffic_signals'],
      dtype='object')

In [13]:
zi.head()

Value
Sensor Name                                   Timestamp        
Newcastle_Cradlewell_Roadside_AURN_AQ_MONITOR 2018-07-01   31.0
                                              2018-07-02   11.0
                                              2018-07-03   17.0
                                              2018-07-04   36.0
                                              2018-07-05   25.0

In [24]:
r2_score(zi,zx['NO2'].median(axis=1))

-7.0969426060706695

In [54]:
from scipy.stats import pearsonr
pearsonr(np.ravel(zi),np.ravel(zx['Temperature'].min(axis=1)))
# np.ravel(zx['day'])

(0.007550134919031691, 0.7049250544672929)

In [188]:
def rfe(zx, zi, n=None, step=1):
    from sklearn.feature_selection import RFE
    from sklearn.ensemble import RandomForestRegressor

    selector = RFE(RandomForestRegressor(), n_features_to_select=n, step=step)
    selector.fit(zx, zi['Value'])
    try:
        sel = pd.DataFrame(columns=zx.columns)
        sel.loc['support']= selector.support_
        sel.loc['ranking'] = selector.ranking_

        cols = sel.loc['support'].reset_index()['support']
        zxtmp = zx.iloc[:,cols[cols].index]
    except:
        zxtmp = selector.transform(zx)
    return zxtmp

In [161]:
zxtmp = rfe(zx,zi)

In [165]:
mlp(RobustScaler().fit_transform(zxtmp), np.log(zi), 50, False)

-0.0324630617177418

In [190]:
def poly_rfe(zx, zi, n=30, step=50):
    from sklearn.preprocessing import PolynomialFeatures
    zxtmp = PolynomialFeatures(degree=2).fit_transform(zx)
    zxtmp = rfe(zxtmp,zi,n=n,step=step)
    return zxtmp2

In [187]:
poly_rfe(zx, zi, n=30, step=50)

(2518, 23)